<a href="https://colab.research.google.com/github/dajoeng/KDT_team4_FinalPJT/blob/main/01_%EC%9B%B9%ED%81%AC%EB%A1%A4%EB%A7%81/%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%9D%B8%EA%B8%B0%EA%B2%80%EC%83%89%EC%96%B4%EC%B6%94%EC%B6%9C_%EB%B0%8F_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 설치

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.3 MB 6.0 MB/s 
     |████████████████████████████████| 140 kB 57.2 MB/s 
     |████████████████████████████████| 384 kB 59.1 MB/s 
     |████████████████████████████████| 58 kB 6.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.13 which is incompatible.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.downlo

In [6]:
# -*- coding: UTF-8 -*-
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
import numpy as np
import itertools
import multiprocessing
from selenium.webdriver.chrome.options import Options

# 밀키트 전체 인기 검색어 가져오기

In [7]:
#인기 검색어 내용 추출
def get_rank_list(driver):
    trend_list = []
    rank_texts = driver.find_elements(by=By.CSS_SELECTOR, value='div.rank_top1000_scroll > ul')
    for rank_text in rank_texts:
        rank_text = rank_text.text
        rank_text = rank_text.split('\n')
        trend_list.extend(rank_text[1::2])
    return trend_list

#인기 검색어 1-40위까지 추출
def get_1_to_40(driver, TIMEOUT):
    trend_1_to_40 = []
    rank_text = get_rank_list(driver) #(1-20)
    trend_1_to_40.extend(rank_text)

    next_btn = driver.find_element(By.CSS_SELECTOR, 'div.top1000_btn_area > div > a.btn_page_next') #(21-40)
    next_btn.click()
    time.sleep(TIMEOUT)
    rank_text = get_rank_list(driver)
    trend_1_to_40.extend(rank_text)
    return trend_1_to_40

#인기 검색어 1-200위까지 추출
def get_1_to_200(driver, TIMEOUT):
    trend_1_to_200 = []
    rank_text = get_rank_list(driver) #(1-20)
    trend_1_to_200.extend(rank_text)

    for cnt in range(9):
      next_btn = driver.find_element(By.CSS_SELECTOR, 'div.top1000_btn_area > div > a.btn_page_next') #(21-200)
      next_btn.click()
      time.sleep(TIMEOUT)
      rank_text = get_rank_list(driver)
      trend_1_to_200.extend(rank_text)

    return trend_1_to_200

In [8]:
# #Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.
# url='https://datalab.naver.com/shoppingInsight/sCategory.naver?cid=50014240'
# TIMEOUT=3

# try:
#   driver = webdriver.ChromeOptions()
#   driver.add_argument('--headless')        # Head-less 설정
#   driver.add_argument('--no-sandbox')
#   driver.add_argument('--disable-dev-shm-usage')
#   driver = webdriver.Chrome('chromedriver', options=driver)
#   driver.get(url)
#   time.sleep(TIMEOUT)
  
#   #전체 밀키트 인기검색어
#   all_trend_list = []
#   all_trend_list.append(get_1_to_200(driver, TIMEOUT))

#   #세부카테고리의 인기 검색어 : '찌개/국','면/파스타','구이','볶음/튀김','조림/찜'
#   for idx in range(2,7):
#       driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[3]/span').click() #select_btn
#       time.sleep(TIMEOUT)
#       driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[3]/ul/li['+str(idx)+']/a').click() #option
#       time.sleep(1.5)
#       driver.find_element(By.CSS_SELECTOR, '#content > div.section_instie_area.space_top > div > div.section.insite_inquiry > div > a').click() #btn_submit
#       time.sleep(1.5)
#       all_trend_list.append(get_1_to_200(driver, 1.5))
        
# except Exception:
#     raise

# finally:
#     if driver is not None:
#         driver.quit()

# 멀티 프로세스

- https://codewnag.tistory.com/26

In [16]:
#------------------------------------------------------------
def do_multi(code):
  print ( str(code)+"번 프로세스 실행", end=', ')
  
  # Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.
  chrome_options = Options()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver', options=chrome_options)

  # 접속
  driver.get(url)
  time.sleep(TIMEOUT)

  #밀키트 전체
  if code == 1:
    driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[1]/span').click() #대분류 select_btn
    time.sleep(TIMEOUT)
    driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[1]/ul/li[7]/a').click() #option
    time.sleep(TIMEOUT)
    driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[2]/span').click() #중분류 select_btn
    time.sleep(TIMEOUT)
    driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[2]/ul/li[22]/a').click() #option
    time.sleep(TIMEOUT)
  #세부카테고리의 인기 검색어 : '찌개/국','면/파스타','구이','볶음/튀김','조림/찜'
  else :
    driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[3]/span').click() #소분류 select_btn
    time.sleep(TIMEOUT)

  if code == 2:
    driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[3]/ul/li[2]/a').click() #option
  elif code == 3:
    driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[3]/ul/li[3]/a').click() #option
  elif code == 4:
    driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[3]/ul/li[4]/a').click() #option
  elif code == 5:
    driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[3]/ul/li[5]/a').click() #option
  elif code == 6:
    driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[3]/ul/li[6]/a').click() #option
  else: 
    pass

  #한번에 적용버튼 클릭 및 트렌드 추출
  time.sleep(TIMEOUT)
  driver.find_element(By.CSS_SELECTOR, '#content > div.section_instie_area.space_top > div > div.section.insite_inquiry > div > a').click() #btn_submit
  time.sleep(TIMEOUT)

  all_trend_list = []
  all_trend_list.append(get_1_to_200(driver, TIMEOUT))

  return all_trend_list

#-----------------------------------------------------------
def main():
  code_list = [id for id in range(1,7)] #창 개수
  print ('--- start _multiprocessing')

  # cpu 갯수 확인
  cpu_count = multiprocessing.cpu_count()
  print ('--- cpu_count ', cpu_count)

  # cpu 수 결정
  pool = multiprocessing.Pool(6)

  # 실행 함수, 넘겨줄 파라미터
  all_trend_list = pool.map(do_multi, code_list)

  # 모든 프로세스 종료까지 기다림
  pool.close()
  pool.join()
  # multi end

  return all_trend_list

In [18]:
#실행코드
if __name__ == '__main__':
  global TIMEOUT, url

  TIMEOUT=1
  url='https://datalab.naver.com/shoppingInsight/sCategory.naver?cid=50014240'
  all_trend_list = main() #200개 : 24초

--- start _multiprocessing
--- cpu_count  2
1번 프로세스 실행, 5번 프로세스 실행4번 프로세스 실행2번 프로세스 실행6번 프로세스 실행, , 3번 프로세스 실행, , , 

## 데이터프레임으로 정리

In [19]:
#데이터프레임으로 정리
detailed_category = ['전체', '찌개/국','면/파스타','구이','볶음/튀김','조림/찜']
trend_df = pd.DataFrame(list(itertools.chain(*all_trend_list))).T
trend_df.columns = detailed_category

%cd /content/drive/My Drive
trend_df.to_csv('네이버데이터랩_인기검색어.csv')

trend_df.info()

/content/drive/My Drive
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   전체      200 non-null    object
 1   찌개/국    200 non-null    object
 2   면/파스타   200 non-null    object
 3   구이      200 non-null    object
 4   볶음/튀김   200 non-null    object
 5   조림/찜    200 non-null    object
dtypes: object(6)
memory usage: 9.5+ KB


In [20]:
trend_df

,전체,찌개/국,면/파스타,구이,볶음/튀김,조림/찜
0,밀키트,밀키트,밀키트,밀키트,밀키트,밀키트
1,떡볶이,낙곱새,동남아식밀키트탄탄면,스테이크밀키트,감바스밀키트,캠핑요리추천
2,낙곱새,알탕,수제비밀키트,자미곱,애슐리밀키트,캠핑음식추천
3,캠핑밀키트,캠핑밀키트,파스타밀키트,목살밀키트,감바스,찜닭밀키트
4,알탕,해신탕,공항칼국수,군고기밀키트,닭갈비밀키트,보일링크랩
...,...,...,...,...,...,...
195,형제소곱창전골,한구곱창전골,바지락술찜,치킨밀키트,곱도리볶음,궁중갈비찜
196,마녀떡볶이3500원,일미만두,빠네크림파스타,대구막창밀키트,딩동쭈꾸미,랜선맛집수제la갈비
197,마라샹궈밀키트,연포탕,호랑이냉면,밀키트찹스테이크,마라샹궈키트,매운쪽갈비
198,소갈비찜밀키트,육개장,마화룡,빕스스테이크,순대밀키트,술안주


# konlpy, Mecab 형태소 분석기 설치 스크립트 실행

## mecab의 폴더를 보면 크게 3개의 폴더로 이뤄져 있습니다.
- mecab-ko-dic : mecab의 단어 사전
- tools : 단어 사전에 단어를 추가시킨 후 컴파일 및 적용을 시켜주는 프로그램들 존재
- user-dic : 사용자가 추가적으로 추가한 단어에 대한 단어 사전

## 단어 사전에 단어를 추가하는 프로세스는 크게 다음과 같이 진행이 됩니다.
1. user-dic의 csv 파일에 형식에 맞게 추가하고 싶은 단어를 추가
2. tools의 add-userdic-win.ps1라는 프로그램을 실행하여 컴파일 진행하여 추가 사항을 적용
3. 2번의 과정에 의해서 user-dic에서 설정한 csv파일이 mecab-ko-dic에 새로 csv파일이 만들어져서 단어 사전에 해당 단어가 적용됨

## mecab-ko-dic 품사 태그 설명
- http://eunjeon.blogspot.com/2014/03/blog-post.html
- https://docs.google.com/spreadsheets/d/1-9blXKjtjeKZqsf4NzHeYJCrr49-nXeRF6D80udfcwY/edit#gid=589544265
- 표층형 / 좌문맥ID / 우문맥ID / 비용(우선순위) /	품사 / 태그 /	의미부류 / 종성 유무 / 	읽기 / 타입 /	첫번째 품사 /	마지막 품사 /	원형 / 인덱스 표현

In [21]:
# konlpy, Mecab 형태소 분석기 설치 스크립트 실행
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab/
!bash ./install_mecab-ko_on_colab190912.sh
%cd /content/mecab-ko-dic-2.1.1-20180720/
!pip install jamo

fatal: destination path 'Mecab-ko-for-Google-Colab' already exists and is not an empty directory.
/content/drive/MyDrive/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 465 kB 49.1 MB/s 
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-12-29 01:07:21--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22e9:9f55, 2406:da00:ff00::22cd:e0db, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?response-content-disposition=attach

# 사용자 단어 사전 추가

In [22]:
from jamo import h2j, j2hcj
%cd /content/mecab-ko-dic-2.1.1-20180720/

def get_nnpFile(word_list):
  #형식에 맞춰서 nnp 사전에 추가
  for word in word_list:
    sample_text_list = list(word)
    last_word = sample_text_list[-1]
    last_word_jamo_list = list(j2hcj(h2j(last_word)))
    last_jamo = last_word_jamo_list[-1]

    ## 마지막 글자가 받침을 가지느냐 안가지느냐로 T는 True, F는 False.
    jongsung_TF = "T"
    if last_jamo in ['ㅏ', 'ㅑ', 'ㅓ', 'ㅕ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ', 'ㅡ', 'ㅣ', 'ㅘ', 'ㅚ', 'ㅙ', 'ㅝ', 'ㅞ', 'ㅢ', 'ㅐ,ㅔ', 'ㅟ', 'ㅖ', 'ㅒ']:
      jongsung_TF = "F"

    #표층형 / 좌문맥ID / 우문맥ID / 비용(우선순위) / 품사 / 태그 / 의미부류 / 종성 유무 / 읽기 / 타입 / 첫번째 품사 / 마지막 품사 / 원형 / 인덱스 표현
    line = '{},,,0,NNP,*,{},{},*,*,*,*,*\n'.format(word, jongsung_TF, word)

    file_data.append(line)

  return file_data

#--------------------------------------------------
with open("./user-dic/nnp.csv", 'r', encoding='utf-8') as f:
  file_data = f.readlines()

#사용자 단어 정의 및 nnp파일에 추가
word_list = ['곱도리탕','소곱창','가니쉬','우족찜','밀키트','투움바','감바스','프레시지','피코크','파스타',
             '된장찌개','마라탕','해신탕','해물탕','낙곱새','낙새','오꼬노미야끼','랍스터','숯불닭갈비',
             '탄탄면','밀푀유나베','술찜','통찜','묵은지찜','김치찜','뽈찜','곱도리','쉬림프','씨푸드',
             '뇨끼','폭립','소꼬리찜','마라샹궈','동남아','에그인헬','빠네','찜닭','해물찜','블랙라벨',
             '오코노미야끼', '막창','리조또','활전복','된장','자미곱','동남아식','더원푸드','맛도령',
             '순살', '쿠킹','한우','소곱창전골','육통령', '된장찌개','당감댁', '캠핑용', '호텔식', '간편',
             '파우즈','레이식당', '팔공산', '얼큰이','허챠밍','돼고비','투다리', '직화', '직화구이',
             '찹스테이크','황토집','솔리보', '정인푸드몰', '고기남자','마이야르','돈이','냄새없는','생생',
             '쿡솜씨', '아히요', '도리명가', '로미랜드', '소제이','하정강', '양푼이','신사강', '파피요트', 
             '등갈비찜', '서민갑부','오징어볶음','쭈꾸미','소갈비찜']

file_data = get_nnpFile(word_list)

with open("./user-dic/nnp.csv", 'w', encoding='utf-8') as f:
  for line in file_data:
    f.write(line)

/content/mecab-ko-dic-2.1.1-20180720


In [23]:
#설정 저장
!bash ./autogen.sh
!make
!sudo make install
!bash ./tools/add-userdic.sh

Looking in current directory for macros.
configure.ac:2: warning: AM_INIT_AUTOMAKE: two- and three-arguments forms are deprecated.  For more info, see:
configure.ac:2: http://www.gnu.org/software/automake/manual/automake.html#Modernize-AM_005fINIT_005fAUTOMAKE-invocation
/bin/bash ./config.status --recheck
running CONFIG_SHELL=/bin/bash /bin/bash ./configure --no-create --no-recursion
checking for a BSD-compatible install... /usr/bin/install -c
checking whether build environment is sane... yes
/content/mecab-ko-dic-2.1.1-20180720/missing: Unknown `--is-lightweight' option
Try `/content/mecab-ko-dic-2.1.1-20180720/missing --help' for more information
configure: WARNING: 'missing' script is too old or missing
checking for a thread-safe mkdir -p... /bin/mkdir -p
checking for gawk... no
checking for mawk... mawk
checking whether make sets $(MAKE)... yes
checking whether make supports nested variables... yes
checking for mecab-config... /usr/local/bin/mecab-config
checking that generated fi

# user nnp 사전에서도 우선순위 적용

In [24]:
nnp = pd.read_csv('user-nnp.csv', index_col = 0)
nnp['3821'] = 0
nnp = nnp.rename(columns={'3821':0})
nnp.to_csv('user-nnp.csv')

* make clean <br>
= 이전 컴파일 내용을 삭제
* make distclean<br>
 = 파일을 받고 압축을 푼 처음상태로 만드는것
* make install<br>
 = make를 통해 만들어진 설치파일(setup)을 설치를 하는 과정이다. 한마디로 build된 프로그램을 실행 할 수 있게 파일들을 알맞은 위치에다가 복사를 한다.

In [25]:
!make clean    #이전 컴파일 내용을 삭제

test -z "model.bin matrix.bin char.bin sys.dic unk.dic" || rm -f model.bin matrix.bin char.bin sys.dic unk.dic


In [26]:
!make install  #만들어진 파일 설치

/usr/local/libexec/mecab/mecab-dict-index -d . -o . -f UTF-8 -t UTF-8
reading ./unk.def ... 13
emitting double-array: 100% |###########################################| 
reading ./Symbol.csv ... 16
reading ./Hanja.csv ... 125750
reading ./XR.csv ... 3637
reading ./J.csv ... 416
reading ./Inflect.csv ... 44820
reading ./EC.csv ... 2547
reading ./XSV.csv ... 23
reading ./NNG.csv ... 208524
reading ./NNB.csv ... 140
reading ./NR.csv ... 482
reading ./VA.csv ... 2360
reading ./MAJ.csv ... 240
reading ./ETN.csv ... 14
reading ./VV.csv ... 7331
reading ./IC.csv ... 1305
reading ./user-nnp.csv ... 92
reading ./Foreign.csv ... 11690
reading ./Person.csv ... 196459
reading ./VCN.csv ... 7
reading ./MAG.csv ... 14242
reading ./CoinedWord.csv ... 148
reading ./user-person.csv ... 1
reading ./NNBC.csv ... 677
reading ./user-place.csv ... 2
reading ./XPN.csv ... 83
reading ./Preanalysis.csv ... 5
reading ./Place.csv ... 30303
reading ./Wikipedia.csv ... 36762
reading ./Place-address.csv ... 19301
r

# 네이버데이터랩 인기검색어 파일에 적용

In [27]:
%cd /content/drive/My Drive

naver_df = trend_df.copy()

/content/drive/My Drive


In [28]:
from konlpy.tag import Mecab
mecab = Mecab()
for column in range(6):
  naver_df.iloc[:, column] = naver_df.iloc[:, column].apply(lambda trend : " ".join(mecab.morphs(trend)))
naver_df.to_csv('네이버데이터랩_인기검색어_전처리.csv')
naver_df

,전체,찌개/국,면/파스타,구이,볶음/튀김,조림/찜
0,밀키트,밀키트,밀키트,밀키트,밀키트,밀키트
1,떡볶이,낙곱새,동남아식 밀키트 탄탄면,스테이크 밀키트,감바스 밀키트,캠핑 요리 추천
2,낙곱새,알탕,수제비 밀키트,자미곱,애슐리 밀키트,캠핑 음식 추천
3,캠핑 밀키트,캠핑 밀키트,파스타 밀키트,목살 밀키트,감바스,찜닭 밀키트
4,알탕,해신탕,공항 칼국수,군고기 밀키트,닭갈비 밀키트,보일링 크랩
...,...,...,...,...,...,...
195,형제 소곱창전골,한 구 곱창전골,바지락 술찜,치킨 밀키트,곱도리 볶음,궁중 갈비찜
196,마녀 떡볶이 3500 원,일미 만 두,빠네 크림 파스타,대구 막창 밀키트,딩동 쭈꾸미,랜 선 맛집 수제 la 갈비
197,마라샹궈 밀키트,연포탕,호랑이 냉면,밀키트 찹스테이크,마라샹궈 키트,매운 쪽 갈비
198,소갈비찜 밀키트,육개장,마화룡,빕스 스테이크,순대 밀키트,술안주


In [29]:
naver_dict = {'찌개/국' : '전골, 생선탕, 찌개/짜글이, 알탕, 해물탕, 국, 마라탕',
         '면/파스타' : '면, 국수, 파스타',
         '구이' : '구이, 스테이크',
         '볶음/튀김' : '볶음, 감바스, 순대',
         '조림/찜' : '찜, 마파두부, 족발',
         '전체' : '한식, 기타, 씨푸드, 닭, 오리, 떡볶이, 무침, 쌈, 샌드위치, 리조또, 카레'}

# 인기 검색어 띄어쓰기 전처리 TEST
- konlpy
- https://heegyukim.medium.com/symspell%EC%9D%84-%EC%9D%B4%EC%9A%A9%ED%95%9C-%ED%95%9C%EA%B8%80-%EB%A7%9E%EC%B6%A4%EB%B2%95-%EA%B5%90%EC%A0%95-2-%EB%B3%B5%ED%95%A9%EC%96%B4%EC%99%80-%EB%9D%84%EC%96%B4%EC%93%B0%EA%B8%B0-%EA%B5%90%EC%A0%95-7605ec2fcebc
- soynlp : https://github.com/lovit/soynlp

In [ ]:
from konlpy.tag import Hannanum, Kkma, Komoran, Okt
hannanum = Hannanum()
kkma = Kkma()
komoran = Komoran()
okt = Okt()

In [ ]:
word = '프리미엄부대찌개밀키트김치2인3인캠핑요리쿠킹박스술안주'
print('hannanum : ', hannanum.morphs(word))
print('kkma : ', kkma.morphs(word))
print('komoran : ', komoran.morphs(word))
print('okt : ', okt.morphs(word))
print('mecab : ', mecab.morphs(word))

In [ ]:
word = '부산맛집밀키트(1회용냄비선물)당감댁곱도리탕닭볶음탕닭도리탕곱창안주캠핑용'
print('hannanum : ', hannanum.morphs(word))
print('kkma : ', kkma.morphs(word))
print('komoran : ', komoran.morphs(word))
print('okt : ', okt.morphs(word))
print('mecab : ', mecab.morphs(word))

In [ ]:
word = '더원푸드시래기곱창전골밀키트'
print('hannanum : ', hannanum.morphs(word))
print('kkma : ', kkma.morphs(word))
print('komoran : ', komoran.morphs(word))
print('okt : ', okt.morphs(word))
print('mecab : ', mecab.morphs(word))

In [ ]:
word = '일품정한우간편안주매콤한우우족찜밀키트650g'
print('hannanum : ', hannanum.morphs(word))
print('kkma : ', kkma.morphs(word))
print('komoran : ', komoran.morphs(word))
print('okt : ', okt.morphs(word))
print('mecab : ', mecab.morphs(word))

In [ ]:
# !pip3 install soynlp
# !wget https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt -O 2016-10-20.txt

In [ ]:
# ### 말 뭉치 생성
# from soynlp import DoublespaceLineCorpus

# # 문서 단위 말뭉치 생성 
# corpus = DoublespaceLineCorpus("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt")
# len(corpus)  # 문서의 갯수

In [ ]:
# import sklearn
# from soynlp.utils import DoublespaceLineCorpus
# from soynlp.noun import LRNounExtractor_v2
# from soynlp import DoublespaceLineCorpus

# corpus_path = '2016-10-20-news'
# sents = DoublespaceLineCorpus(corpus_path, iter_sent=True)

# noun_extractor = LRNounExtractor_v2(verbose=True)
# nouns = noun_extractor.train_extract(sents)
